In [1]:
import pandas as pd
import json
from ast import literal_eval
import numpy as np

In [2]:
with open(f'data/female/female-layered-tops.json') as f:
    female_layered_tops = json.load(f)
with open(f'data/male/male-layered-tops.json') as f:
    male_layered_tops = json.load(f)

In [3]:
df_female_style_clothing_type=pd.read_csv('data_combinations/female_style_clothing_transcript.csv',index_col=0)
df_male_style_clothing_type=pd.read_csv('data_combinations/male_style_clothing_transcript.csv',index_col=0)
female_layered_tops=pd.DataFrame(female_layered_tops)
male_layered_tops=pd.DataFrame(male_layered_tops)
df_temperatures=pd.read_csv('data/type_temperature_intervals.csv')

In [98]:
def f(gender_style_clothing_type,
      gender_layered_tops,
      type_temperature):
    gender_layered_tops=gender_layered_tops.explode('top_list').reset_index(drop=True)
    #get info about type transcripts
    result=gender_layered_tops.merge(gender_style_clothing_type[['type','transcriptTypeParams']],left_on='layered_top_type',right_on='type')\
    .merge(gender_style_clothing_type[['type','transcriptTypeParams']],left_on='top_list',right_on='type')\
    .groupby(['transcriptTypeParams_x'],as_index=False)['transcriptTypeParams_y']\
    .apply(list).rename(columns={'transcriptTypeParams_x':'layered_top_type','transcriptTypeParams_y':'top_list'})
    #create df with missing types
    all_types_style_clothing = gender_style_clothing_type[gender_style_clothing_type['category'].isin(['top','outerwear'])]['type'].unique().tolist()
    all_types_layered = gender_layered_tops.layered_top_type.unique().tolist()
    missing_types_layered = [ element for element in all_types_style_clothing if element not in all_types_layered] 
    missing_types_layered_df =gender_style_clothing_type[gender_style_clothing_type['type'].isin(missing_types_layered)][['type','transcriptTypeParams']]['transcriptTypeParams']\
        .to_frame()\
        .rename(columns={'transcriptTypeParams':'layered_top_type'})\
        .reset_index(drop=True)
    missing_types_layered_df['top_list']= np.nan
    #filter by styles
    
    # working with temperatures
    temperature_df=type_temperature.merge(gender_style_clothing_type[['category','type','transcriptTypeParams']],left_on='clothingType',right_on='type')\
                        .drop(columns=['type'])
    
    return pd.concat([result,missing_types_layered_df]).reset_index(drop=True)


In [7]:
z=f(df_female_style_clothing_type,female_layered_tops)

In [9]:
df_temperatures=pd.read_csv('data/type_temperature_intervals.csv')

In [37]:
x=df_temperatures.merge(df_female_style_clothing_type[['category','type','transcriptTypeParams']],left_on='clothingType',right_on='type').drop(columns=['type'])

In [53]:
x2=x[x['clothingType']=='sweatshirt'].loc[0,['temp_m40_m10','temp_m10_m0','temp_m0_p10','temp_p10_p20','temp_p20']].values

In [86]:
x[(x['temp_m40_m10']==x2[0])|
  (x['temp_m10_m0']==x2[1])|
  (x['temp_m0_p10']==x2[2])|
  (x['temp_p10_p20']==x2[3])|
  (x['temp_p20']==x2[4])]\
[x['category']=='bottom']['transcriptTypeParams'].tolist()

['waist-regular length-cropped joggers',
 'waist-regular length-regular joggers',
 'waist-high length-cropped joggers',
 'waist-high length-regular joggers',
 'waist-regular fit-tapered sweatpants',
 'waist-regular fit-straight sweatpants',
 'waist-regular fit-wide sweatpants',
 'waist-high fit-tapered sweatpants',
 'waist-high fit-straight sweatpants',
 'waist-high fit-wide sweatpants',
 'waist-regular fit-tapered length-cropped classic_trousers',
 'waist-regular fit-tapered length-regular classic_trousers',
 'waist-regular fit-straight length-cropped classic_trousers',
 'waist-regular fit-straight length-regular classic_trousers',
 'waist-regular fit-wide length-cropped classic_trousers',
 'waist-regular fit-wide length-regular classic_trousers',
 'waist-regular fit-flared length-cropped classic_trousers',
 'waist-regular fit-flared length-regular classic_trousers',
 'waist-high fit-tapered length-cropped classic_trousers',
 'waist-high fit-tapered length-regular classic_trousers',
 

In [96]:
# filter data for category
def similar_tops_by_temperature(top_type:str,
       temperature_df:pd.DataFrame,
       category:str):
    list_of_temperatures=temperature_df[temperature_df['clothingType']==top_type]\
                        .reset_index(drop=True)\
                        .loc[0,['temp_m40_m10','temp_m10_m0','temp_m0_p10','temp_p10_p20','temp_p20']].values
    result_list=temperature_df[(temperature_df['temp_m40_m10']==list_of_temperatures[0])|
    (temperature_df['temp_m10_m0']==list_of_temperatures[1])|
    (temperature_df['temp_m0_p10']==list_of_temperatures[2])|
    (temperature_df['temp_p10_p20']==list_of_temperatures[3])|
    (temperature_df['temp_p20']==list_of_temperatures[4])]\
    [temperature_df['category']==category]['transcriptTypeParams'].tolist()
    return result_list


In [ ]:
need to find any intersections between types and temp regimes

In [94]:
x[x['clothingType']=='hoodie'].loc[['temp_m40_m10','temp_m10_m0','temp_m0_p10','temp_p10_p20','temp_p20']]

,clothingType,temp_m40_m10,temp_m10_m0,temp_m0_p10,temp_p10_p20,temp_p20,category,transcriptTypeParams
4,hoodie,0,0,0,1,0,top,length-cropped fit-regular hoodie
5,hoodie,0,0,0,1,0,top,length-cropped fit-oversize hoodie
6,hoodie,0,0,0,1,0,top,length-regular fit-regular hoodie
7,hoodie,0,0,0,1,0,top,length-regular fit-oversize hoodie


In [97]:
similar_tops_by_temperature('hoodie',x,'bottom')

['waist-regular length-cropped joggers',
 'waist-regular length-regular joggers',
 'waist-high length-cropped joggers',
 'waist-high length-regular joggers',
 'waist-regular fit-tapered sweatpants',
 'waist-regular fit-straight sweatpants',
 'waist-regular fit-wide sweatpants',
 'waist-high fit-tapered sweatpants',
 'waist-high fit-straight sweatpants',
 'waist-high fit-wide sweatpants',
 'waist-regular fit-tapered length-cropped classic_trousers',
 'waist-regular fit-tapered length-regular classic_trousers',
 'waist-regular fit-straight length-cropped classic_trousers',
 'waist-regular fit-straight length-regular classic_trousers',
 'waist-regular fit-wide length-cropped classic_trousers',
 'waist-regular fit-wide length-regular classic_trousers',
 'waist-regular fit-flared length-cropped classic_trousers',
 'waist-regular fit-flared length-regular classic_trousers',
 'waist-high fit-tapered length-cropped classic_trousers',
 'waist-high fit-tapered length-regular classic_trousers',
 

In [46]:
x[(x['category']=='footwear')]

,clothingType,temp_m40_m10,temp_m10_m0,temp_m0_p10,temp_p10_p20,temp_p20,category,transcriptTypeParams
197,low-top_sneakers,0,1,1,1,1,footwear,low-top_sneakers
198,high-top_sneakers,0,1,1,1,0,footwear,high-top_sneakers
199,boots,1,1,1,0,0,footwear,heel_type-stiletto boots
200,boots,1,1,1,0,0,footwear,heel_type-platform boots
201,boots,1,1,1,0,0,footwear,heel_type-heel boots
202,boots,1,1,1,0,0,footwear,heel_type-flat boots
203,loafers,0,0,0,1,1,footwear,loafers
322,booties,0,0,1,1,0,footwear,heel_type-stiletto booties
323,booties,0,0,1,1,0,footwear,heel_type-platform booties
324,booties,0,0,1,1,0,footwear,heel_type-heel booties


In [2]:
with open('data/female-types.json') as f:
    female_types = json.load(f)
with open('data/male-types.json') as f:
    male_types = json.load(f)
    

In [3]:
female_types_df=pd.DataFrame(female_types)
male_types_df=pd.DataFrame(male_types)

In [4]:
types_df=pd.concat([male_types_df,female_types_df])

In [5]:
all_types=types_df.drop_duplicates().reset_index(drop=True)

In [10]:
all_types['type'].unique().tolist()

['sweatshirt',
 'hoodie',
 'turtleneck',
 'sweater',
 'cardigan',
 'vest',
 't-shirt',
 'long_sleeve_t-shirt',
 'polo',
 'short_sleeve_shirt',
 'long_sleeve_shirt',
 'denim_shirt',
 'linen_shirt',
 'blazer',
 'joggers',
 'sweatpants',
 'classic_trousers',
 'casual_trousers',
 'cargo',
 'linen_trousers',
 'jeans',
 'casual_shorts',
 'denim_shorts',
 'leather_jacket',
 'down_jacket',
 'denim_jacket',
 'bomber_jacket',
 'sleeveless_jacket',
 'windbreaker',
 'parka',
 'demi-season_jacket',
 'coat',
 'trench_coat',
 'low-top_sneakers',
 'high-top_sneakers',
 'boots',
 'chelsea_boots',
 'chukka_boots',
 'uggs_boots',
 'oxford_shoes',
 'dress_shoes',
 'loafers',
 'short_sleeve_blouse',
 'long_sleeve_blouse',
 'sleeveless_blouse',
 'linen_blouse',
 'crop_top',
 'sleeveless_top',
 'leggings',
 'leather_trousers',
 'bermuda_shorts',
 'leather_shorts',
 'leather_skirt',
 'denim_skirt',
 'pleated_skirt',
 'knitted_skirt',
 'fur_coat',
 'booties',
 'ankle_boots',
 'pumps',
 'dress_sandals',
 'flats

In [14]:
types_df[types_df['category']=='footwear']

,category,subcategory,type
33,footwear,sneakers,low-top_sneakers
34,footwear,sneakers,high-top_sneakers
35,footwear,boots,boots
36,footwear,boots,chelsea_boots
37,footwear,boots,chukka_boots
38,footwear,boots,uggs_boots
39,footwear,dress_shoes,oxford_shoes
40,footwear,dress_shoes,dress_shoes
41,footwear,loafers,loafers
46,footwear,sneakers,low-top_sneakers


In [18]:
llm_output = [{ 'winter': [ 'down_jacket', 'parka', 'coat', 'fur_coat', 'boots', 'chelsea_boots', 'chukka_boots', 'uggs_boots', 'ankle_boots', 'booties' ],
 'spring': [ 't-shirt', 'polo', 'short_sleeve_shirt', 'linen_shirt', 'sweatpants', 'classic_trousers', 'casual_trousers', 'cargo', 'linen_trousers', 'jeans', 'casual_shorts', 'denim_shorts', 'sleeveless_jacket', 'windbreaker', 'demi-season_jacket', 'trench_coat', 'low-top_sneakers', 'high-top_sneakers', 'loafers', 'short_sleeve_blouse', 'linen_blouse', 'crop_top', 'sleeveless_top', 'leggings', 'bermuda_shorts', 'leather_trousers', 'denim_skirt', 'pleated_skirt', 'knitted_skirt', 'flats', 'dress_sandals', 'oxford_shoes', 'dress_shoes', 'leather_jacket', 'denim_jacket', 'bomber_jacket' ], 
 'summer': [ 't-shirt', 'short_sleeve_shirt', 'linen_shirt', 'casual_shorts', 'denim_shorts', 'sleeveless_jacket', 'low-top_sneakers', 'high-top_sneakers', 'short_sleeve_blouse', 'linen_blouse', 'crop_top', 'sleeveless_top', 'bermuda_shorts', 'leather_shorts', 'leather_skirt', 'denim_skirt', 'pleated_skirt', 'knitted_skirt', 'dress_sandals', 'flats', 'loafers', 'pumps' ], 
 'autumn': [ 'sweatshirt', 'hoodie', 'turtleneck', 'sweater', 'cardigan', 'vest', 'long_sleeve_t-shirt', 'long_sleeve_shirt', 'denim_shirt', 'blazer', 'joggers', 'sweatpants', 'classic_trousers', 'casual_trousers', 'cargo', 'linen_trousers', 'jeans', 'leather_trousers', 'leggings', 'boots', 'chelsea_boots', 'chukka_boots', 'ankle_boots', 'booties', 'loafers', 'long_sleeve_blouse', 'sleeveless_blouse', 'linen_blouse', 'oxford_shoes', 'dress_shoes', 'trench_coat', 'leather_jacket', 'denim_jacket', 'bomber_jacket' ] }]

In [25]:
seasons_ref=pd.DataFrame(llm_output).T.explode(0).reset_index().rename(columns={'index':'seasons',0:'types'})
seasons_ref['types'].drop_duplicates()

In [38]:
all_types=all_types.merge(seasons_ref,left_on='type',right_on='types',how='left')

In [40]:
def match_season(text):
    if text =='summer':
        return 1
    if text =='spring':
        return 2
    if text =='autumn':
        return 3
    if text =='winter':
        return 4

In [45]:
all_types['seasons_numb']=all_types['seasons'].apply(match_season)

In [50]:
def match_season2(text):
    if text == 1:
        return 'summer'
    if text == 2:
        return 'spring'
    if text == 3:
        return 'autumn'
    if text == 4:
        return 'winter'

In [51]:
all_types=all_types.groupby(['category','subcategory','type'],as_index=False)['seasons_numb'].max()
all_types['seasons']=all_types['seasons_numb'].apply(match_season2)

In [53]:
all_types.to_csv('data_combinations/types-season.csv')